In [2]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.insert(0, '../scripts/')

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 196

In [4]:
%load_ext autoreload
%autoreload 1

In [5]:
%aimport shapeslib
%aimport etllib
%aimport smoothlib

In [7]:
s000 = pd.read_csv('../source/income_mountain/0000.csv')
s000.head()

,isInterpolated,useMicroData,CountryCode,CountryName,RegionCode,CoverageType,RequestYear,DataYear,DataType,PPP,...,Decile1,Decile2,Decile3,Decile4,Decile5,Decile6,Decile7,Decile8,Decile9,Decile10
0,0,1,ALB,Albania,XX,N,1981,1996.00,X,54.652580,...,0.038630,0.05289,0.06379,0.07322,0.08380,0.09355,0.10820,0.1247,0.1490,0.2122
1,0,0,DZA,Algeria,XX,N,1981,1988.00,X,31.517970,...,0.027000,0.03839,0.04869,0.05868,0.06898,0.08035,0.09398,0.1124,0.1434,0.3281
2,0,1,AGO,Angola,XX,N,1981,2000.21,X,80.931800,...,0.009832,0.02195,0.03342,0.04495,0.05663,0.07049,0.08808,0.1135,0.1587,0.4025
3,1,1,ARG,Argentina,XX,U,1981,NaN,Y,3.161164,...,-1.000000,-1.00000,-1.00000,-1.00000,-1.00000,-1.00000,-1.00000,-1.0000,-1.0000,-1.0000
4,0,0,ARM,Armenia,XX,N,1981,1996.00,Y,165.629000,...,0.022960,0.03236,0.04186,0.05187,0.06291,0.07575,0.09185,0.1144,0.1538,0.3522


In [40]:
s000["Mean"]

0       220.77280
1       240.87570
2       166.38240
3       671.63370
4       285.08040
          ...    
6640    348.77690
6641    328.30880
6642     70.69390
6643     87.84403
6644    121.60830
Name: Mean, Length: 6645, dtype: float64

In [10]:
s000[["CountryCode", "CoverageType", "RequestYear", "Median"]]

,CountryCode,CoverageType,RequestYear,Median
0,ALB,N,1981,194.02750
1,DZA,N,1981,179.23560
2,AGO,N,1981,105.24240
3,ARG,U,1981,-1.00000
4,ARM,N,1981,196.52880
...,...,...,...,...
6640,VNM,N,2019,283.00250
6641,PSE,N,2019,271.20730
6642,YEM,N,2019,54.11205
6643,ZMB,N,2019,45.81299


In [11]:
df = s000[["CountryCode", "CoverageType", "RequestYear", "Median"]].copy()

In [12]:
dfg = df.groupby(['CountryCode', 'CoverageType'])

In [22]:
dfg.get_group(('SWE', 'N')).set_index('RequestYear')['Median'].replace(-1, np.nan).interpolate() * 0.035

RequestYear
1981    28.834285
1982    29.065622
1983    29.296959
1984    29.528296
1985    29.759633
1986    29.990970
1987    30.222308
1988    31.281159
1989    32.340011
1990    33.398862
1991    34.457714
1992    35.516565
1993    35.026138
1994    34.535711
1995    34.045284
1996    34.708608
1997    35.371932
1998    36.035257
1999    36.698581
2000    37.361905
2001    37.651413
2002    37.940922
2003    38.230430
2004    38.485860
2005    39.463970
2006    41.362965
2007    43.459255
2008    44.383430
2009    45.163930
2010    45.776010
2011    46.385885
2012    47.262845
2013    48.055035
2014    49.346500
2015    50.559390
2016    51.372720
2017    51.408525
2018    51.879870
2019    51.973040
Name: Median, dtype: float64

In [116]:
def run(x):
    res = (x.set_index('RequestYear')['Median']
            .replace(-1, np.nan)
            .fillna(method='backfill')
            .fillna(method='ffill')) 
    return res * 0.035

In [117]:
res = dfg.apply(run)

In [52]:
na = res[pd.isnull(res)].copy()

In [44]:
df2 = s000[["CountryCode", "CoverageType", "RequestYear", "Mean"]].copy()

In [45]:
dfg2 = df2.groupby(['CountryCode', 'CoverageType'])

In [50]:
dfg2.get_group(('CHN', 'A')).set_index('RequestYear')['Mean'] * 0.02

RequestYear
1981    0.699189
1982    0.752171
1983    0.820199
1984    0.928573
1985    1.006182
1986    1.061228
1987    1.139731
1988    1.251810
1989    1.096326
1990    1.127408
1991    1.167375
1992    1.235868
1993    1.313318
1994    1.435557
1995    1.549732
1996    1.662871
1997    1.703331
1998    1.753375
1999    1.828017
2000    2.009114
2001    2.137562
2002    2.320350
2003    2.479378
2004    2.698678
2005    3.017828
2006    3.214124
2007    3.499030
2008    3.716426
2009    4.003486
2010    4.367160
2011    4.685048
2012    5.118096
2013    5.757128
2014    6.202240
2015    6.639226
2016    7.076768
2017    7.762478
2018    8.427046
2019    8.964766
Name: Mean, dtype: float64

In [56]:
na.reset_index()['CountryCode'].unique()

array(['CHN', 'IDN', 'IND', 'SOM'], dtype=object)

In [57]:
res

CountryCode  CoverageType  RequestYear
AGO          N             1981           3.683484
                           1982           3.554912
                           1983           3.574403
                           1984           3.656202
                           1985           3.652268
                                            ...   
ZWE          N             2015           2.718764
                           2016           2.718764
                           2017           2.718764
                           2018           2.718764
                           2019           2.718764
Name: Median, Length: 6645, dtype: float64

In [60]:
def run2(x):
    res = (x.set_index('RequestYear')['Mean']
            .replace(-1, np.nan)
            .fillna(method='backfill')
            .fillna(method='ffill')) 
    return res * 0.02

In [61]:
res2 = dfg2.apply(run2)

In [62]:
res2

CountryCode  CoverageType  RequestYear
AGO          N             1981           3.327648
                           1982           3.211496
                           1983           3.229108
                           1984           3.303004
                           1985           3.299448
                                            ...   
ZWE          N             2015           2.690096
                           2016           2.489078
                           2017           2.382992
                           2018           2.573732
                           2019           2.432166
Name: Mean, Length: 6645, dtype: float64

In [141]:
res0 = res.fillna(res2)

In [102]:
res0.loc[('IND', 'A')]

year
1981    1.282867
1982    1.277302
1983    1.304930
1984    1.316434
1985    1.352521
1986    1.370852
1987    1.395210
1988    1.441232
1989    1.450788
1990    1.459290
1991    1.460885
1992    1.463789
1993    1.472468
1994    1.484749
1995    1.504629
1996    1.533333
1997    1.540128
1998    1.564773
1999    1.588524
2000    1.599021
2001    1.624199
2002    1.633224
2003    1.660653
2004    1.684845
2005    1.726931
2006    1.755084
2007    1.803112
2008    1.830872
2009    1.864820
2010    1.966041
2011    2.147570
2012    2.221650
2013    2.311630
2014    2.394050
2015    2.505478
2016    2.626752
2017    2.733970
2018    2.733970
2019    2.733970
Name: Median, dtype: float64

In [142]:
res0.index.names = ['country', 'coverage', 'time']
res0.name = 'poverty_line'

In [143]:
res0

country  coverage  time
AGO      N         1981    3.683484
                   1982    3.554912
                   1983    3.574403
                   1984    3.656202
                   1985    3.652268
                             ...   
ZWE      N         2015    2.718764
                   2016    2.718764
                   2017    2.718764
                   2018    2.718764
                   2019    2.718764
Name: poverty_line, Length: 6645, dtype: float64

In [144]:
res0[pd.isnull(res0)]

Series([], Name: poverty_line, dtype: float64)

In [145]:
res0 = res0.reset_index()

In [146]:
res0['country'] = res0['country'].replace({'XKX': 'KOS'})

In [147]:
res0['country'] = res0['country'].str.lower()

In [125]:
res0.coverage.unique()

array(['N', 'U', 'A', 'R'], dtype=object)

In [111]:
def run3(x):
    for c in list('NAUR'):
        if c in x['coverage'].unique():
            return x[x.coverage == c].drop(columns=['coverage'])
    raise ValueError('bang!')

In [148]:
res_final = res0.groupby('country').apply(run3).reset_index(drop=True)

In [127]:
res_final

,country,year,poverty_line
0,ago,1981,3.683484
1,ago,1982,3.554912
2,ago,1983,3.574403
3,ago,1984,3.656202
4,ago,1985,3.652268
...,...,...,...
6406,zwe,2015,2.718764
6407,zwe,2016,2.718764
6408,zwe,2017,2.718764
6409,zwe,2018,2.718764


In [128]:
res_final[res_final.country == 'aus']

,country,year,poverty_line
195,aus,1981,30.665142
196,aus,1982,31.164749
197,aus,1983,31.164749
198,aus,1984,31.164749
199,aus,1985,31.164749
200,aus,1986,31.967842
201,aus,1987,31.967842
202,aus,1988,31.967842
203,aus,1989,31.967842
204,aus,1990,29.412229


In [88]:
from ddf_utils.str import format_float_digits
from functools import partial

In [149]:
res_final['poverty_line'] = res_final['poverty_line'].map(format_float_digits)

In [150]:
res_final.to_csv('../../poverty_rates/ddf--datapoints--poverty_line--by--country--time.csv', index=False)